In [1]:
import tensorflow as tf
import os, warnings
import pandas as pd
warnings.filterwarnings('ignore')
import numpy as np
from logp.zq.io.smi2feat import load_feat_from_csv
from logp.zq.io.load_dataframe import load_pd
from logp.zq.model.model import potential_model
from sklearn.metrics import mean_absolute_error,mean_squared_error
from logp.zq.io.base import sparse_batch

In [2]:
filename = 'logp/DATASETS/sf_ave_PhysProp_MD_weights_charge.csv'
dataset = pd.read_csv(filename)
dataset.shape

(4802, 4)

In [3]:
results = load_pd(filename)

In [4]:
# load the predicting data in martel
filename_martel = 'logp/DATASETS/Martel/MD_ave/weight_q/sf_ave_Martel_MD_weights_charge.csv'
dataset_martel = pd.read_csv(filename_martel)
dataset_martel.shape
results_martel = load_pd(filename_martel)
dataset_martel = lambda: load_feat_from_csv(results_martel,24,split={'train':10,'test':0},shuffle=False)
martel_train = lambda: dataset_martel()['train'].apply(sparse_batch(5))

In [5]:
# load the predicting data in star
filename_star = 'logp/DATASETS/star_nonstar/MD_ave/weight_q/sf_ave_star_MD_weights_charge.csv'
dataset_star = pd.read_csv(filename_star)
dataset_star.shape
results_star = load_pd(filename_star)

dataset_star = lambda: load_feat_from_csv(results_star,24,split={'train':10,'test':0},shuffle=False)
star_train = lambda: dataset_star()['train'].apply(sparse_batch(5))

In [6]:
# load the predicting data in huusk
filename_nonstar = 'logp/DATASETS/Huuskonen/MD_ave/weight_q/sf_ave_Huuskonen_MD_weights_charge.csv'
dataset_nonstar = pd.read_csv(filename_nonstar)
dataset_nonstar.shape
results_nonstar = load_pd(filename_nonstar)

dataset_nonstar = lambda: load_feat_from_csv(results_nonstar,24,split={'train':10,'test':0},shuffle=False)
nonstar_train = lambda: dataset_nonstar()['train'].apply(sparse_batch(5))

In [7]:
logps_martel = []
logps_star = []
logps_nonstar = []

for x in range(10):
    # define the parameters
    params = {
    'model_dir':'MD_ave_weight_q_PhysProp/100_100_100/lr_1e-4/kfold_{}'.format(x),
    'network':'bpnn',
    'network_params':{
    'nn_spec':{1:[100,100,100],6:[100,100,100],7:[100,100,100],8:[100,100,100]}},
    'model_params': {'learning_rate': 1e-4}
    }
    #split the dataset into train and test
    ftrain = 'logp/DATASETS/PhysProps/MD_ave/weight_q/sf_ave_PhysProp_MD_weights_q_train_{}.csv'.format(x)
    ftest  = 'logp/DATASETS/PhysProps/MD_ave/weight_q/sf_ave_PhysProp_MD_weights_q_test_{}.csv'.format(x)
    dtrain = load_pd(ftrain)
    dtest = load_pd(ftest)
    dataset_train = lambda: load_feat_from_csv(dtrain,24,split={'train':10,'test':0})
    dataset_test = lambda: load_feat_from_csv(dtest,24,split={'train':0,'test':10})
    # yeild the train and test `input_fn`
    train = lambda: dataset_train()['train'].repeat().shuffle(1000).apply(sparse_batch(100))
    test = lambda: dataset_test()['test'].repeat().apply(sparse_batch(50))
    
    # initialize the model
    model = potential_model(params)
    # define the hooks
    early_stopping = tf.estimator.experimental.stop_if_no_decrease_hook(model, metric_name='loss',
                                                              max_steps_without_decrease=1000,
                                                              min_steps=100)
    # train and evaluate the model
    train_spec = tf.estimator.TrainSpec(input_fn=train, hooks=[early_stopping] ,max_steps=200000)
    eval_spec = tf.estimator.EvalSpec(input_fn=test, steps=400)
    tf.estimator.train_and_evaluate(model, train_spec, eval_spec)
    
    # predict the data in martel 
    preds_martel = model.predict(input_fn=martel_train)
    preds_star = model.predict(input_fn=star_train)
    preds_nonstar = model.predict(input_fn=nonstar_train)
    
    logp_martel = []
    for pred in preds_martel:
        logp_martel.append(pred['logp'])
    logps_martel.append(logp_martel)
    
    logp_star = []
    for pred in preds_star:
        logp_star.append(pred['logp'])
    logps_star.append(logp_star)
    
    logp_nonstar = []
    for pred in preds_nonstar:
        logp_nonstar.append(pred['logp'])
    logps_nonstar.append(logp_nonstar)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'MD_ave_weight_q_PhysProp/100_100_100/lr_1e-4/kfold_0', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Not using Distribute Coordina

INFO:tensorflow:loss = 0.44767, step = 4900 (0.964 sec)
INFO:tensorflow:global_step/sec: 101.955
INFO:tensorflow:loss = 0.48725092, step = 5000 (0.981 sec)
INFO:tensorflow:global_step/sec: 103.915
INFO:tensorflow:loss = 0.40683845, step = 5100 (0.962 sec)
INFO:tensorflow:global_step/sec: 102.318
INFO:tensorflow:loss = 0.67555135, step = 5200 (0.977 sec)
INFO:tensorflow:global_step/sec: 102.503
INFO:tensorflow:loss = 0.482496, step = 5300 (0.976 sec)
INFO:tensorflow:global_step/sec: 102.39
INFO:tensorflow:loss = 0.37394202, step = 5400 (0.977 sec)
INFO:tensorflow:global_step/sec: 103.139
INFO:tensorflow:loss = 0.64322424, step = 5500 (0.971 sec)
INFO:tensorflow:global_step/sec: 103.372
INFO:tensorflow:loss = 0.37666443, step = 5600 (0.966 sec)
INFO:tensorflow:global_step/sec: 102.2
INFO:tensorflow:loss = 0.59398633, step = 5700 (0.979 sec)
INFO:tensorflow:global_step/sec: 103.542
INFO:tensorflow:loss = 0.63505644, step = 5800 (0.966 sec)
INFO:tensorflow:global_step/sec: 102.29
INFO:tens

INFO:tensorflow:global_step/sec: 102.031
INFO:tensorflow:loss = 0.21812943, step = 13200 (0.980 sec)
INFO:tensorflow:global_step/sec: 102.622
INFO:tensorflow:loss = 0.30346963, step = 13300 (0.975 sec)
INFO:tensorflow:global_step/sec: 102.87
INFO:tensorflow:loss = 0.39978355, step = 13400 (0.972 sec)
INFO:tensorflow:global_step/sec: 102.622
INFO:tensorflow:loss = 0.6008404, step = 13500 (0.974 sec)
INFO:tensorflow:global_step/sec: 102.368
INFO:tensorflow:loss = 0.544148, step = 13600 (0.977 sec)
INFO:tensorflow:global_step/sec: 102.555
INFO:tensorflow:loss = 0.2614914, step = 13700 (0.976 sec)
INFO:tensorflow:global_step/sec: 102.482
INFO:tensorflow:loss = 0.2858704, step = 13800 (0.976 sec)
INFO:tensorflow:global_step/sec: 103.124
INFO:tensorflow:loss = 0.33517772, step = 13900 (0.969 sec)
INFO:tensorflow:global_step/sec: 102.659
INFO:tensorflow:loss = 0.6384856, step = 14000 (0.974 sec)
INFO:tensorflow:global_step/sec: 102.534
INFO:tensorflow:loss = 0.34144452, step = 14100 (0.976 se

INFO:tensorflow:global_step/sec: 102.242
INFO:tensorflow:loss = 0.48278567, step = 21400 (0.978 sec)
INFO:tensorflow:global_step/sec: 103.487
INFO:tensorflow:loss = 0.3173174, step = 21500 (0.966 sec)
INFO:tensorflow:global_step/sec: 103.38
INFO:tensorflow:loss = 0.4052006, step = 21600 (0.968 sec)
INFO:tensorflow:global_step/sec: 102.666
INFO:tensorflow:loss = 0.2737977, step = 21700 (0.974 sec)
INFO:tensorflow:global_step/sec: 103.119
INFO:tensorflow:loss = 0.5851136, step = 21800 (0.970 sec)
INFO:tensorflow:global_step/sec: 102.622
INFO:tensorflow:loss = 0.2532691, step = 21900 (0.974 sec)
INFO:tensorflow:global_step/sec: 102.35
INFO:tensorflow:loss = 0.22261748, step = 22000 (0.977 sec)
INFO:tensorflow:global_step/sec: 102.657
INFO:tensorflow:loss = 0.21194822, step = 22100 (0.974 sec)
INFO:tensorflow:global_step/sec: 103.08
INFO:tensorflow:loss = 0.26724964, step = 22200 (0.970 sec)
INFO:tensorflow:global_step/sec: 102.182
INFO:tensorflow:loss = 0.42137355, step = 22300 (0.979 sec

INFO:tensorflow:global_step/sec: 102.345
INFO:tensorflow:loss = 0.22149926, step = 29600 (0.977 sec)
INFO:tensorflow:global_step/sec: 102.363
INFO:tensorflow:loss = 0.26820016, step = 29700 (0.977 sec)
INFO:tensorflow:global_step/sec: 102.59
INFO:tensorflow:loss = 0.23886833, step = 29800 (0.974 sec)
INFO:tensorflow:global_step/sec: 103.774
INFO:tensorflow:loss = 0.41054398, step = 29900 (0.964 sec)
INFO:tensorflow:global_step/sec: 102.588
INFO:tensorflow:loss = 0.37235656, step = 30000 (0.975 sec)
INFO:tensorflow:global_step/sec: 102.131
INFO:tensorflow:loss = 0.38688976, step = 30100 (0.979 sec)
INFO:tensorflow:global_step/sec: 104.01
INFO:tensorflow:loss = 0.19062571, step = 30200 (0.961 sec)
INFO:tensorflow:global_step/sec: 102.721
INFO:tensorflow:loss = 0.24572657, step = 30300 (0.974 sec)
INFO:tensorflow:global_step/sec: 103.305
INFO:tensorflow:loss = 0.23319058, step = 30400 (0.968 sec)
INFO:tensorflow:global_step/sec: 102.527
INFO:tensorflow:loss = 0.2976446, step = 30500 (0.97

...

INFO:tensorflow:global_step/sec: 105.593
INFO:tensorflow:loss = 0.07916712, step = 166800 (0.947 sec)
INFO:tensorflow:global_step/sec: 106.8
INFO:tensorflow:loss = 0.09261115, step = 166900 (0.936 sec)
INFO:tensorflow:global_step/sec: 106.678
INFO:tensorflow:loss = 0.07930116, step = 167000 (0.937 sec)
INFO:tensorflow:global_step/sec: 105.172
INFO:tensorflow:loss = 0.13251124, step = 167100 (0.951 sec)
INFO:tensorflow:global_step/sec: 105.454
INFO:tensorflow:loss = 0.06291305, step = 167200 (0.949 sec)
INFO:tensorflow:global_step/sec: 105.966
INFO:tensorflow:loss = 0.0922854, step = 167300 (0.943 sec)
INFO:tensorflow:global_step/sec: 105.549
INFO:tensorflow:loss = 0.17010656, step = 167400 (0.947 sec)
INFO:tensorflow:global_step/sec: 106.384
INFO:tensorflow:loss = 0.08349512, step = 167500 (0.941 sec)
INFO:tensorflow:global_step/sec: 106.073
INFO:tensorflow:loss = 0.0677748, step = 167600 (0.942 sec)
INFO:tensorflow:global_step/sec: 105.804
INFO:tensorflow:loss = 0.07423161, step = 167

INFO:tensorflow:loss = 0.06596477, step = 174800 (0.940 sec)
INFO:tensorflow:global_step/sec: 106.001
INFO:tensorflow:loss = 0.045111675, step = 174900 (0.943 sec)
INFO:tensorflow:global_step/sec: 105.749
INFO:tensorflow:loss = 0.13177311, step = 175000 (0.946 sec)
INFO:tensorflow:global_step/sec: 106.13
INFO:tensorflow:loss = 0.058386024, step = 175100 (0.942 sec)
INFO:tensorflow:global_step/sec: 105.822
INFO:tensorflow:loss = 0.0812891, step = 175200 (0.945 sec)
INFO:tensorflow:global_step/sec: 106.452
INFO:tensorflow:loss = 0.0645649, step = 175300 (0.939 sec)
INFO:tensorflow:global_step/sec: 104.842
INFO:tensorflow:loss = 0.062107686, step = 175400 (0.954 sec)
INFO:tensorflow:global_step/sec: 106.808
INFO:tensorflow:loss = 0.05401719, step = 175500 (0.936 sec)
INFO:tensorflow:global_step/sec: 105.749
INFO:tensorflow:loss = 0.046035375, step = 175600 (0.946 sec)
INFO:tensorflow:global_step/sec: 105.611
INFO:tensorflow:loss = 0.07233401, step = 175700 (0.947 sec)
INFO:tensorflow:glob

INFO:tensorflow:global_step/sec: 105.864
INFO:tensorflow:loss = 0.069649294, step = 182900 (0.945 sec)
INFO:tensorflow:global_step/sec: 105.363
INFO:tensorflow:loss = 0.060368247, step = 183000 (0.949 sec)
INFO:tensorflow:global_step/sec: 105.521
INFO:tensorflow:loss = 0.054670487, step = 183100 (0.948 sec)
INFO:tensorflow:global_step/sec: 106.218
INFO:tensorflow:loss = 0.07277675, step = 183200 (0.942 sec)
INFO:tensorflow:global_step/sec: 105.48
INFO:tensorflow:loss = 0.056357134, step = 183300 (0.948 sec)
INFO:tensorflow:global_step/sec: 105.121
INFO:tensorflow:loss = 0.0958008, step = 183400 (0.951 sec)
INFO:tensorflow:global_step/sec: 105.485
INFO:tensorflow:loss = 0.04693918, step = 183500 (0.948 sec)
INFO:tensorflow:global_step/sec: 105.82
INFO:tensorflow:loss = 0.06483839, step = 183600 (0.945 sec)
INFO:tensorflow:global_step/sec: 106.132
INFO:tensorflow:loss = 0.06669983, step = 183700 (0.942 sec)
INFO:tensorflow:global_step/sec: 104.021
INFO:tensorflow:loss = 0.11690672, step 

INFO:tensorflow:Evaluation [160/400]
INFO:tensorflow:Evaluation [200/400]
INFO:tensorflow:Evaluation [240/400]
INFO:tensorflow:Evaluation [280/400]
INFO:tensorflow:Evaluation [320/400]
INFO:tensorflow:Evaluation [360/400]
INFO:tensorflow:Evaluation [400/400]
INFO:tensorflow:Inference Time : 2.14333s
INFO:tensorflow:Finished evaluation at 2021-09-22-21:07:30
INFO:tensorflow:Saving dict for global step 190192: METRICS/E_LOSS = 0.24353848, METRICS/E_MAE = 0.3567021, METRICS/E_MSE = 0.24353848, METRICS/E_RMSE = 0.49349618, METRICS/TOT_LOSS = 0.2435385, global_step = 190192, loss = 0.2435385
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 190192: MD_ave_weight_q_PhysProp/100_100_100/lr_1e-4/kfold_9/model.ckpt-190192
INFO:tensorflow:global_step/sec: 28.1614
INFO:tensorflow:loss = 0.077804625, step = 190200 (3.551 sec)
INFO:tensorflow:global_step/sec: 106.005
INFO:tensorflow:loss = 0.0618867, step = 190300 (0.943 sec)
INFO:tensorflow:global_step/sec: 105.169
INFO:tensorflow:l

INFO:tensorflow:global_step/sec: 107.271
INFO:tensorflow:loss = 0.03805146, step = 197500 (0.932 sec)
INFO:tensorflow:global_step/sec: 106.652
INFO:tensorflow:loss = 0.032751225, step = 197600 (0.938 sec)
INFO:tensorflow:global_step/sec: 106.575
INFO:tensorflow:loss = 0.058404304, step = 197700 (0.938 sec)
INFO:tensorflow:global_step/sec: 105.318
INFO:tensorflow:loss = 0.03836204, step = 197800 (0.949 sec)
INFO:tensorflow:global_step/sec: 106.244
INFO:tensorflow:loss = 0.053669788, step = 197900 (0.942 sec)
INFO:tensorflow:global_step/sec: 106.72
INFO:tensorflow:loss = 0.04442489, step = 198000 (0.936 sec)
INFO:tensorflow:global_step/sec: 106.393
INFO:tensorflow:loss = 0.07457163, step = 198100 (0.940 sec)
INFO:tensorflow:global_step/sec: 105.998
INFO:tensorflow:loss = 0.06351096, step = 198200 (0.943 sec)
INFO:tensorflow:global_step/sec: 106.368
INFO:tensorflow:loss = 0.056761652, step = 198300 (0.940 sec)
INFO:tensorflow:global_step/sec: 106.28
INFO:tensorflow:loss = 0.044419203, ste

In [8]:
# average the results
logps_martel = np.array(logps_martel)
logps_martel_ave = np.mean(logps_martel,axis=0)
#logps_martel_ave

logps_star = np.array(logps_star)
logps_star_ave = np.mean(logps_star,axis=0)

logps_nonstar = np.array(logps_nonstar)
logps_nonstar_ave = np.mean(logps_nonstar, axis=0)

In [9]:
# get the true value of martel
logp_exp_martel  = []
for x in results_martel:
    logp_exp_martel.append(x[3])

# get the true value of star
logp_exp_star = []
for x in results_star:
    logp_exp_star.append(x[3])
    
# get the true value of nonstar
logp_exp_nonstar = []
for x in results_nonstar:
    logp_exp_nonstar.append(x[3])

In [10]:
# calculate the mae, mse, and rmse of martel
mae_martel = mean_absolute_error(logps_martel_ave,logp_exp_martel)
mse_martel = mean_squared_error(logps_martel_ave,logp_exp_martel)
mae_martel,mse_martel,np.sqrt(mse_martel)

(0.9806308312898271, 1.7987089358551949, 1.3411595489930328)

In [11]:
# calculate the mae, mse, and rmse of star
mae_star = mean_absolute_error(logps_star_ave,logp_exp_star)
mse_star = mean_squared_error(logps_star_ave,logp_exp_star)
mae_star,mse_star,np.sqrt(mse_star)

(0.6026683363576348, 0.7755898367315235, 0.8806757841178122)

In [12]:
# calculate the mae, mse, and rmse of nonstar
mae_nonstar = mean_absolute_error(logps_nonstar_ave,logp_exp_nonstar)
mse_nonstar = mean_squared_error(logps_nonstar_ave,logp_exp_nonstar)
mae_nonstar,mse_nonstar,np.sqrt(mse_nonstar)

(0.2306092954220652, 0.13548971228702078, 0.3680892721705168)